[🥭 Mango Markets](https://mango.markets/) support is available at: [Docs](https://docs.mango.markets/) | [Discord](https://discord.gg/67jySBhxrg) | [Twitter](https://twitter.com/mangomarkets) | [Github](https://github.com/blockworks-foundation) | [Email](mailto:hello@blockworks.foundation)

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/blockworks-foundation/mango-explorer-examples/HEAD?labpath=ImmediateBuyAndSell.ipynb) [Run this code](https://mybinder.org/v2/gh/blockworks-foundation/mango-explorer-examples/HEAD?labpath=ImmediateBuyAndSell.ipynb) on Binder.

_🏃‍♀️ To run this notebook press the ⏩ icon in the toolbar above._

# 🥭 Immediate Buy and Sell

This notebook shows how to buy and sell from the current orderbook, rather than placing orders to be executed later.

This example will SELL 1 SOL on SOL/USDC and then BUY it back.

The code uses 'Immediate Or Cancel' (IOC) orders. Orders of this type never appear on the orderbook - they exist to explicitly 'take' orders from the orderbook. If it's not possible to fulfill the IOC order quantity at the specified price, it is cancelled.


In [ ]:
import decimal
import mango


# Use this function to display balances because it always fetches fresh data
def show_balances(context, wallet, group):
    # Find the wrapped SOL token
    sol_token = context.instrument_lookup.find_by_symbol_or_raise("SOL")

    # Find the current SPL token balance
    token_account = mango.TokenAccount.fetch_largest_for_owner_and_token(context, wallet.address, sol_token)
    print("Wrapped SOL in wallet:", token_account.value)

    # Find the SOL-specific details in the account
    accounts = mango.Account.load_all_for_owner(context, wallet.address, group)
    for slot in accounts[0].slots:
        if slot.base_instrument == sol_token:
            print("SOL in account:       ", slot.net_value)
            print("USDC in account:      ", accounts[0].shared_quote.net_value)
            print()
            return


# Use our hard-coded devnet wallet for DeekipCw5jz7UgQbtUbHQckTYGKXWaPQV4xY93DaiM6h.
# For real-world use you'd load the bytes from the environment or a file.
wallet = mango.Wallet(bytes([67,218,68,118,140,171,228,222,8,29,48,61,255,114,49,226,239,89,151,110,29,136,149,118,97,189,163,8,23,88,246,35,187,241,107,226,47,155,40,162,3,222,98,203,176,230,34,49,45,8,253,77,136,241,34,4,80,227,234,174,103,11,124,146]))

# Create a 'devnet' Context
with mango.ContextBuilder.build(cluster_name="devnet") as context:
    # Load the wallet's account
    group = mango.Group.load(context)
    accounts = mango.Account.load_all_for_owner(context, wallet.address, group)
    account = accounts[0]

    max_slippage = decimal.Decimal("0.005")  # Allow at most 0.5% slippage from top-of-book
    quantity = decimal.Decimal(1)

    print("Initial balances before trades:")
    show_balances(context, wallet, group)

    market_operations = mango.operations(context, wallet, account, "SOL/USDC", dry_run=False)

    sell_signatures = market_operations.market_sell(quantity, max_slippage)
    print("IOC SELL transaction signatures:", sell_signatures)

    print("Waiting for sell transaction to confirm...\n")
    mango.WebSocketTransactionMonitor.wait_for_all(
            context.client.cluster_ws_url, sell_signatures, commitment="processed"
        )

    show_balances(context, wallet, group)

    buy_signatures = market_operations.market_buy(quantity, max_slippage)
    print("IOC BUY transaction signatures:", buy_signatures)

    print("Waiting for buy transaction to confirm...\n")
    mango.WebSocketTransactionMonitor.wait_for_all(
            context.client.cluster_ws_url, buy_signatures, commitment="processed"
        )

    show_balances(context, wallet, group)

print("Example complete.")